![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

# Introduction to `VTiles` for WherobotsDB

In this notebook we will create PMTiles vector tiles for rendering maps using building and road data from the Overture Maps dataset in the Wherobots Open Data Catalog.

## Introduction To Vector Tiles

Vector tiles provide performant rendering of map data for large vector feature datasets across large regions and zoom
levels. Here’s why, and when, they should be used:

* Vector tiles are designed for use in web maps, mobile apps, and desktop GIS software.
* WherobotsDB makes it easy and affordable to generate vector tiles at a planetary scale.
* By rendering vector tiles directly, the interactive map experience is more responsive and scalable for large datasets
  than rendering feature formats (e.g., GeoJSON) directly and allows developers to customize the display, which is
  otherwise impossible with raster tiles.

## How To Generate Vector Tiles

### Start a Sedona Session

As always, begin by starting a Sedona context

In [ ]:
import org.apache.sedona.spark.SedonaContext

val config = SedonaContext.builder().getOrCreate()
val sedona = SedonaContext.create(config)

### Load Feature Data

Create a Spatial DataFrame with a geometry column and a layer column. The geometry column contains the features to render in the
map. The layer column is a string that describes the grouping the feature should be in. Records within the same layer
can be styled together, independently of other layers. In this case example features that represent buildings are in the buildings layer and those representing roads are in the roads layer.

The first cell that follows gives some variable to control where we generate tiles for. The default is a small town in Washington: Issaquah.

In [ ]:
import org.apache.spark.sql.sedona_sql.expressions.st_constructors.ST_GeomFromText
import org.apache.spark.sql.sedona_sql.expressions.st_predicates.ST_Intersects
import org.apache.spark.sql.functions.{lit, col}

// Set to False to generate tiles for the entire dataset, true to generate only for regionWkt area
val filter = true
val regionWkt = "POLYGON ((-122.097931 47.538528, -122.048836 47.566566, -121.981888 47.510012, -122.057076 47.506302, -122.097931 47.538528))"
val filterExpression = ST_Intersects(col("geometry"), ST_GeomFromText(lit(regionWkt)))

Next, we create the buildings Spatial DataFrame using the Overture buildings table from the Wherobots Open Data Catalog.

In [ ]:
import org.apache.spark.sql.functions.element_at

val buildingsDf = sedona.table("wherobots_open_data.overture_2024_02_15.buildings_building")
  .select(
    col("geometry"),
    lit("buildings").alias("layer"),
    element_at(col("sources"), 1)("dataset").alias("source")
  )

buildingsDf.show()

Next, we create a Spatial DataFrame for our road features using the Overture transportation segment table.

In [ ]:
val roadsDf = sedona.table("wherobots_open_data.overture_2024_02_15.transportation_segment")
  .select(
    col("geometry"),
    lit("roads").alias("layer"),
    element_at(col("sources"), 1)("dataset").alias("source")
  )

roadsDf.show()

Next, we prepare a single spatial DataFrame combining our roads and buildings features.

In [ ]:
var featuresDf = roadsDf.union(buildingsDf)

featuresDf = if (filter) featuresDf.filter(filterExpression) else featuresDf

featuresDf.count()

### Create DataFrame of Tiles

Once we have the Spatial DataFrame ready for tile generation, we can use the `VTiles.generate` method to create a DataFrame containing the encoded tiles. A `GenerationConfig` object can optionally be provided as a second argument if a non-default configuration is desired.

In [ ]:
import com.wherobots.VTiles

val tilesDf = VTiles.generate(featuresDf)

tilesDf.show(3, 150, true)

### Write Tiles to File Storage

WherobotsDB creates vector tiles in the PMTiles format by default. The PMTiles format is a performant, simple, and
optimized format for creating vector tiles and provides an interface for configuration through a `PMTilesConfig` object.
In this example, we leverage a WherobotsDB feature to configure the archive directly from the feature DataFrame used to
generated the tiles:

In [ ]:
val fullTilesPath = sys.env("USER_S3_PATH") + "tiles.pmtiles"
VTiles.writePMTiles(tilesDf, fullTilesPath, featuresDf = Some(featuresDf))

## Quick Generation of Tiles

Sometimes you want to quickly visualize a massive dataset. To achieve this goal, WherobotsDB provides functionality for
quickly generating and saving tiles. When testing this function it completed 100 million features in
less than 5 minutes on a Wherobots Cloud Cairo runtime. This is accomplished by limiting the features processed to 100 million and
generating fewer zoom levels at a higher resolution. At high zooms, the low precision from the low maximum zoom may be
evident.

The Scala/Java API exposes the `getQuickConfig` method which can be passed to
the `vtiles.generate` or `vtiles.generatePMTiles` methods for the same tile generation functionality.

This feature can be used as follows:

In [ ]:
val SampleTilesPath = sys.env("USER_S3_PATH") + "sampleTiles.pmtiles"
VTiles.generatePMTiles(featuresDf, SampleTilesPath, VTiles.getQuickConfig)

As a comprehensive map application toolbox, WherobotsDB provides many off-the-shelf scalable tools. In this
tutorial, we just focus on a minimum example. Detailed explanation of each tool can be found
in [the documentation](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Overview/).